In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Input
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

2024-05-07 09:54:16.624967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 09:54:16.625126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 09:54:16.794948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
tweets = train_data['text'].values
labels = train_data['target'].values
x_train, x_val, y_train, y_val = train_test_split(tweets, labels, test_size=0.3, random_state=42, shuffle=True, stratify=None)
x_val, x_test1, y_val, y_test1 = train_test_split(x_val, y_val, test_size=0.1, random_state=42, shuffle=True, stratify=None)

In [4]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweets)
x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen=350)
x_val = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(x_val, maxlen=350)
x_test1 = tokenizer.texts_to_sequences(x_test1)
x_test1 = pad_sequences(x_test1, maxlen=350)
model = Sequential()
model.add(Input((x_train.shape[1],)))
model.add(Embedding(5000, 100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(150, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l1(0.001)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 350, 100)       │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 350, 100)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           151 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650,751 (2.48 MB)

 Trainable params: 650,751 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.fit(x_train, y_train, epochs=5, batch_size=44, validation_data=(x_val, y_val))

Epoch 1/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 65s 506ms/step - accuracy: 0.5976 - loss: 0.6781 - val_accuracy: 0.7324 - val_loss: 0.5516
Epoch 2/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 61s 500ms/step - accuracy: 0.7739 - loss: 0.4988 - val_accuracy: 0.8083 - val_loss: 0.4759
Epoch 3/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 82s 501ms/step - accuracy: 0.8397 - loss: 0.3996 - val_accuracy: 0.8044 - val_loss: 0.4619
Epoch 4/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 61s 501ms/step - accuracy: 0.8686 - loss: 0.3399 - val_accuracy: 0.7976 - val_loss: 0.4764
Epoch 5/5
122/122 ━━━━━━━━━━━━━━━━━━━━ 61s 501ms/step - accuracy: 0.8898 - loss: 0.2931 - val_accuracy: 0.7873 - val_loss: 0.5118


In [6]:
y_pred = model.predict(x_test1)
y_pred = np.rint(y_pred)
print('Accuracy: {:.3f}'.format(accuracy_score(y_test1, y_pred)))
print("F1 Score:", f1_score(y_test1, y_pred))
test_data = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test_data.isna().any()
test_data
tweets = test_data['text']
x_test = tokenizer.texts_to_sequences(tweets)
x_test = pad_sequences(x_test, maxlen=350)
y_pred = model.predict(x_test)
y_pred = np.rint(y_pred)
y_pred
df_result = pd.DataFrame(test_data['id'], columns=['id'])
df_result['target'] = y_pred.astype(int)
df_result

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step
Accuracy: 0.812
F1 Score: 0.77720207253886
102/102 ━━━━━━━━━━━━━━━━━━━━ 14s 136ms/step


,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [7]:
df_result.to_csv("/kaggle/working/submission.csv", index=False)